In [1]:
import os
import pandas as pd
from tensorflow.keras.utils import to_categorical
from numpy import mean, std, dstack
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from sklearn.utils import resample, class_weight
from sklearn.metrics import f1_score, recall_score, confusion_matrix,precision_score
from sklearn.utils import class_weight
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score, precision_score



# define the parameter values
epochs, batch_size = 5, 128  # Reduced number of epochs and increased batch size
print('Number of epochs: ', epochs)
print('Batch size: ', batch_size)

# load the dataset, returns train and test X and y elements
def load_dataset(path):
    # Load input data and output data
    print('\nLoading dataset...')
    files = os.listdir(path)
    dataX_files = [file for file in files if 'x_' in file]
    datay_files = [file for file in files if 'y_' in file]
    dataX_files.sort()
    datay_files.sort()

    dataX = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in dataX_files])
    
    datay = pd.DataFrame()  # Initialize datay as an empty DataFrame
    unique_label = 1
    for file in datay_files:
        datay_temp = pd.read_csv(path + '/' + file, delimiter=',')
        if (datay_temp.iloc[:, 0] == 1).any():  # Check if there are any 1 values in the first column
            datay_temp.loc[datay_temp.iloc[:, 0] == 1, datay_temp.columns[0]] = unique_label
            if datay.empty:  # If datay is empty, directly assign datay_temp
                datay = datay_temp
            else:
                datay = pd.concat([datay, datay_temp])  # Append only the modified datay_temp
            unique_label += 1

    # Split data into train and test sets
    trainX, testX = model_selection.train_test_split(dataX, test_size=0.10, random_state=42, shuffle=True)
    trainy, testy = model_selection.train_test_split(datay, test_size=0.10, random_state=42, shuffle=True)

    # Get 3D training data (assuming your data requires this step)
    listtrain = [trainX]
    listtest = [testX]
    trainX = dstack(listtrain)
    testX = dstack(listtest)
    print('\nAfter stacking...')
    print('trainX.shape: ', trainX.shape)
    print('testX.shape: ', testX.shape)

    # Calculate number of classes based on unique values in trainy
    num_classes = np.unique(trainy).shape[0]

    # Convert output data to categorical form
    trainy = to_categorical(trainy, num_classes=num_classes)
    testy = to_categorical(testy, num_classes=num_classes)
    print('\nAfter categorizing...')
    print('trainy.shape: ', trainy.shape)
    print('testy.shape: ', testy.shape, '\n')

    return trainX, trainy, testX, testy

from sklearn.metrics import f1_score, recall_score, confusion_matrix,precision_score
from sklearn.utils import class_weight
from sklearn.metrics import precision_score
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import AveragePooling1D
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import class_weight
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Conv1D, AveragePooling1D, MaxPooling1D, Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.utils import resample
import numpy as np

from sklearn.metrics import f1_score

import time

def evaluate_model(trainX, trainy, testX, testy):
    n_features, n_added_dimension, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

    # Define your learning rates
    learning_rates = [0.0005]

    for lr in learning_rates:
        print(f'Training model with learning rate: {lr}')

        # Define the model
        model = Sequential()
        model.add(LSTM(100, input_shape=(n_features, n_added_dimension), return_sequences=True))
        model.add(Dropout(0.2))
        model.add(Conv1D(64, 3, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(50, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=lr), metrics=['accuracy'])

        # Use early stopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=5)

        # Measure training time
        start_time = time.time()
        model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=1, callbacks=[early_stopping])
        training_time = time.time() - start_time

        # Measure testing time
        start_time = time.time()
        _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
        testing_time = time.time() - start_time
        
        # Predict the test set results
        y_pred = model.predict(testX, batch_size=batch_size)
       

        # Convert testy from one-hot encoded to single-digit class labels
        testy_single_digit = np.argmax(testy, axis=1)
        y_pred_single_digit = np.argmax(y_pred, axis=1)
    
        # Now calculate the metrics
        accuracy = accuracy_score(testy_single_digit, y_pred_single_digit)
        f1 = f1_score(testy_single_digit, y_pred_single_digit, average='weighted')
        recall = recall_score(testy_single_digit, y_pred_single_digit, average='weighted')
        precision = precision_score(testy_single_digit, y_pred_single_digit, average='weighted')
        cm = confusion_matrix(testy_single_digit, y_pred_single_digit)

        print(f'Accuracy : {accuracy}')
        print(f'F1 Score: {f1}')
        print(f'Precision: {precision}')
        print(f'Recall: {recall}')
        print(f'Confusion Matrix: \n{cm}\n') 
        print(f'Training time: {training_time} seconds')
        print(f'Testing time: {testing_time} seconds')

    return accuracy, f1, recall, cm





# summarize scores

    
def run_experiment(repeats, datasetname):
    experiment_start_time = time.time()
    trainX, trainy, testX, testy = load_dataset(datasetname)
    print('Data shapes:', trainX.shape, trainy.shape, testX.shape, testy.shape)  # Debugging line
    print('Evaluating for ' + datasetname + '...\n')
    evaluation_start_time = time.time()
    scores = list()
    for r in range(repeats):
        print('Repeat: ', r + 1) 
        score = evaluate_model(trainX, trainy, testX, testy)
        print('Model output length:', len(score))  # Corrected line
        
        
     
    
    print("Duration of loading data: %d seconds" % (evaluation_start_time - experiment_start_time))
    print("Duration of evaluating model: %d seconds" % (time.time() - evaluation_start_time))
    print("Duration of whole experiment: %d seconds" % (time.time() - experiment_start_time))

run_experiment(1, 'C:\\Users\\nalla\\OneDrive\\Provision_PT_838\\dataset\\Data')


Number of epochs:  5
Batch size:  128

Loading dataset...

After stacking...
trainX.shape:  (1557162, 115, 1)
testX.shape:  (173018, 115, 1)

After categorizing...
trainy.shape:  (1557162, 11)
testy.shape:  (173018, 11) 

Data shapes: (1557162, 115, 1) (1557162, 11) (173018, 115, 1) (173018, 11)
Evaluating for C:\Users\nalla\OneDrive\Provision_PT_838\dataset\Data...

Repeat:  1
Training model with learning rate: 0.0005


C:\Users\nalla\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
12166/12166 ━━━━━━━━━━━━━━━━━━━━ 1526s 125ms/step - accuracy: 0.9093 - loss: 0.1779
Epoch 2/5
    1/12166 ━━━━━━━━━━━━━━━━━━━━ 35:24 175ms/step - accuracy: 0.9141 - loss: 0.1761

C:\Users\nalla\anaconda3\Lib\site-packages\keras\src\callbacks\early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


12166/12166 ━━━━━━━━━━━━━━━━━━━━ 1409s 116ms/step - accuracy: 0.9374 - loss: 0.1040
Epoch 3/5
12166/12166 ━━━━━━━━━━━━━━━━━━━━ 1532s 126ms/step - accuracy: 0.9391 - loss: 0.1010
Epoch 4/5
12166/12166 ━━━━━━━━━━━━━━━━━━━━ 1617s 133ms/step - accuracy: 0.9400 - loss: 0.0985
Epoch 5/5
12166/12166 ━━━━━━━━━━━━━━━━━━━━ 1713s 141ms/step - accuracy: 0.9407 - loss: 0.0970
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 75s 55ms/step - accuracy: 0.9415 - loss: 0.0949
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 67s 50ms/step
Accuracy : 0.9414627379810193
F1 Score: 0.9232200724575946
Precision: 0.9657742436726245
Recall: 0.9414627379810193
Confusion Matrix: 
[[98528     0     0     0     0     1     0     0     0     0     0]
 [    0  5639     3     1     0     0     0     0     0     0     0]
 [    0     3  2898     0     0     0     0     0     0     0     0]
 [    1     0     0  2817     0     0     0     0     0     0     1]
 [    1     0     0     0    12  8941     0     0     0     0     0]
 [    1     0     0     0     

In [2]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.utils import to_categorical
from numpy import mean, std, dstack
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score, precision_score

# load the dataset, returns train and test X and y elements
def load_dataset(path):
    # Load input data and output data
    print('\nLoading dataset...')
    files = os.listdir(path)
    dataX_files = [file for file in files if 'x_' in file]
    datay_files = [file for file in files if 'y_' in file]
    dataX_files.sort()
    datay_files.sort()

    dataX = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in dataX_files])
    
    datay = pd.DataFrame()  # Initialize datay as an empty DataFrame
    unique_label = 1
    for file in datay_files:
        datay_temp = pd.read_csv(path + '/' + file, delimiter=',')
        if (datay_temp.iloc[:, 0] == 1).any():  # Check if there are any 1 values in the first column
            datay_temp.loc[datay_temp.iloc[:, 0] == 1, datay_temp.columns[0]] = unique_label
            if datay.empty:  # If datay is empty, directly assign datay_temp
                datay = datay_temp
            else:
                datay = pd.concat([datay, datay_temp])  # Append only the modified datay_temp
            unique_label += 1

    # Split data into train and test sets
    trainX, testX = model_selection.train_test_split(dataX, test_size=0.10, random_state=42, shuffle=True)
    trainy, testy = model_selection.train_test_split(datay, test_size=0.10, random_state=42, shuffle=True)

    # Get 3D training data (assuming your data requires this step)
    listtrain = [trainX]
    listtest = [testX]
    trainX = dstack(listtrain)
    testX = dstack(listtest)
    print('\nAfter stacking...')
    print('trainX.shape: ', trainX.shape)
    print('testX.shape: ', testX.shape)

    # Calculate number of classes based on unique values in trainy
    num_classes = np.unique(trainy).shape[0]

    # Convert output data to categorical form
    trainy = to_categorical(trainy.values.ravel(), num_classes=num_classes)
    testy = to_categorical(testy.values.ravel(), num_classes=num_classes)
    print('\nAfter categorizing...')
    print('trainy.shape: ', trainy.shape)
    print('testy.shape: ', testy.shape, '\n')

    return trainX, trainy, testX, testy

from sklearn.metrics import f1_score, recall_score, confusion_matrix,precision_score
from sklearn.utils import class_weight
from sklearn.metrics import precision_score

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import AveragePooling1D

from sklearn.neighbors import KNeighborsClassifier

def evaluate_model(trainX, trainy, testX, testy):
    # Flatten the data for KNN
    trainX_flat = trainX.reshape(trainX.shape[0], -1)
    testX_flat = testX.reshape(testX.shape[0], -1)

    # Convert categorical labels back to 1-D for KNN
    trainy_flat = np.argmax(trainy, axis=1)
    testy_flat = np.argmax(testy, axis=1)

    # Define the model with n_neighbors=3
    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(trainX_flat, trainy_flat)

    # Predict the test set results
    y_pred = model.predict(testX_flat)

    # Calculate Accuracy, F1 score, Recall, and Precision
    accuracy = accuracy_score(testy_flat, y_pred)
    f1 = f1_score(testy_flat, y_pred, average='weighted')
    recall = recall_score(testy_flat, y_pred, average='weighted')
    precision = precision_score(testy_flat, y_pred, average='weighted')

    # Compute confusion matrix
    cm = confusion_matrix(testy_flat, y_pred)

    print(f'Accuracy : {accuracy}')
    print(f'F1 Score: {f1}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'Confusion Matrix: \n{cm}\n')       

    return accuracy, f1, recall, precision, cm

  
def run_experiment(repeats, datasetname):
    experiment_start_time = time.time()
    trainX, trainy, testX, testy = load_dataset(datasetname)
    print('Data shapes:', trainX.shape, trainy.shape, testX.shape, testy.shape)  # Debugging line
    print('Evaluating for ' + datasetname + '...\n')
    evaluation_start_time = time.time()
    scores = list()
    for r in range(repeats):
        print('Repeat: ', r + 1) 
        score = evaluate_model(trainX, trainy, testX, testy)
        print('Model output length:', len(score))  # Corrected line
        accuracy= score[0] * 100
        print('Score: %.2f' % accuracy, '\n')
        scores.append(score)
    print('Results for ' + datasetname + ':')
    
    print("Duration of loading data: %d seconds" % (evaluation_start_time - experiment_start_time))
    print("Duration of evaluating model: %d seconds" % (time.time() - evaluation_start_time))
    print("Duration of whole experiment: %d seconds" % (time.time() - experiment_start_time))

run_experiment(1, 'C:\\Users\\nalla\\OneDrive\\Provision_PT_838\\dataset\\Data')




Loading dataset...

After stacking...
trainX.shape:  (1557162, 115, 1)
testX.shape:  (173018, 115, 1)

After categorizing...
trainy.shape:  (1557162, 11)
testy.shape:  (173018, 11) 

Data shapes: (1557162, 115, 1) (1557162, 11) (173018, 115, 1) (173018, 11)
Evaluating for C:\Users\nalla\OneDrive\Provision_PT_838\dataset\Data...

Repeat:  1
Accuracy : 0.9675004912783641
F1 Score: 0.9679186233750645
Recall: 0.9675004912783641
Precision: 0.9687541300694644
Confusion Matrix: 
[[98529     0     0     0     0     0     0     0     0     0     0]
 [   20  5623     0     0     0     0     0     0     0     0     0]
 [    4     3  2894     0     0     0     0     0     0     0     0]
 [   48     4     1  2759     0     0     1     5     1     0     0]
 [   10     3     0     1  8940     0     0     0     0     0     0]
 [    6     3     0     1     0 10398     0     0     0     0     0]
 [   36     1     0     2     0     0  4601     0  1179   251    59]
 [   59     0     0    10     0     0  

In [3]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.utils import to_categorical
from numpy import mean, std, dstack
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score, precision_score

# load the dataset, returns train and test X and y elements
def load_dataset(path):
    # Load input data and output data
    print('\nLoading dataset...')
    files = os.listdir(path)
    dataX_files = [file for file in files if 'x_' in file]
    datay_files = [file for file in files if 'y_' in file]
    dataX_files.sort()
    datay_files.sort()

    dataX = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in dataX_files])
    
    datay = pd.DataFrame()  # Initialize datay as an empty DataFrame
    unique_label = 1
    for file in datay_files:
        datay_temp = pd.read_csv(path + '/' + file, delimiter=',')
        if (datay_temp.iloc[:, 0] == 1).any():  # Check if there are any 1 values in the first column
            datay_temp.loc[datay_temp.iloc[:, 0] == 1, datay_temp.columns[0]] = unique_label
            if datay.empty:  # If datay is empty, directly assign datay_temp
                datay = datay_temp
            else:
                datay = pd.concat([datay, datay_temp])  # Append only the modified datay_temp
            unique_label += 1

    # Split data into train and test sets
    trainX, testX = model_selection.train_test_split(dataX, test_size=0.10, random_state=42, shuffle=True)
    trainy, testy = model_selection.train_test_split(datay, test_size=0.10, random_state=42, shuffle=True)

    # Get 3D training data (assuming your data requires this step)
    listtrain = [trainX]
    listtest = [testX]
    trainX = dstack(listtrain)
    testX = dstack(listtest)
    print('\nAfter stacking...')
    print('trainX.shape: ', trainX.shape)
    print('testX.shape: ', testX.shape)

    # Calculate number of classes based on unique values in trainy
    num_classes = np.unique(trainy).shape[0]

    # Convert output data to categorical form
    trainy = to_categorical(trainy.values.ravel(), num_classes=num_classes)
    testy = to_categorical(testy.values.ravel(), num_classes=num_classes)
    print('\nAfter categorizing...')
    print('trainy.shape: ', trainy.shape)
    print('testy.shape: ', testy.shape, '\n')

    return trainX, trainy, testX, testy

from sklearn.metrics import f1_score, recall_score, confusion_matrix,precision_score
from sklearn.utils import class_weight
from sklearn.metrics import precision_score

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import AveragePooling1D

from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import AdaBoostClassifier

def evaluate_model(trainX, trainy, testX, testy):
    # Flatten the data for AdaBoost
    trainX_flat = trainX.reshape(trainX.shape[0], -1)
    testX_flat = testX.reshape(testX.shape[0], -1)

    # Convert categorical labels back to 1-D for AdaBoost
    trainy_flat = np.argmax(trainy, axis=1)
    testy_flat = np.argmax(testy, axis=1)

    # Define the model with n_estimators=100
    model = AdaBoostClassifier(n_estimators=100)
    model.fit(trainX_flat, trainy_flat)

    # Predict the test set results
    y_pred = model.predict(testX_flat)

    # Calculate Accuracy, F1 score, Recall, and Precision
    accuracy = accuracy_score(testy_flat, y_pred)
    f1 = f1_score(testy_flat, y_pred, average='weighted')
    recall = recall_score(testy_flat, y_pred, average='weighted')
    precision = precision_score(testy_flat, y_pred, average='weighted')

    # Compute confusion matrix
    cm = confusion_matrix(testy_flat, y_pred)

    print(f'Accuracy : {accuracy}')
    print(f'F1 Score: {f1}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'Confusion Matrix: \n{cm}\n')       

    return accuracy, f1, recall, precision, cm


  
def run_experiment(repeats, datasetname):
    experiment_start_time = time.time()
    trainX, trainy, testX, testy = load_dataset(datasetname)
    print('Data shapes:', trainX.shape, trainy.shape, testX.shape, testy.shape)  # Debugging line
    print('Evaluating for ' + datasetname + '...\n')
    evaluation_start_time = time.time()
    scores = list()
    for r in range(repeats):
        print('Repeat: ', r + 1) 
        score = evaluate_model(trainX, trainy, testX, testy)
        print('Model output length:', len(score))  # Corrected line
        accuracy= score[0] * 100
        print('Score: %.2f' % accuracy, '\n')
        scores.append(score)
    print('Results for ' + datasetname + ':')
    
    print("Duration of loading data: %d seconds" % (evaluation_start_time - experiment_start_time))
    print("Duration of evaluating model: %d seconds" % (time.time() - evaluation_start_time))
    print("Duration of whole experiment: %d seconds" % (time.time() - experiment_start_time))

run_experiment(1, 'C:\\Users\\nalla\\OneDrive\\Provision_PT_838\\dataset\\Data')


Loading dataset...

After stacking...
trainX.shape:  (1557162, 115, 1)
testX.shape:  (173018, 115, 1)

After categorizing...
trainy.shape:  (1557162, 11)
testy.shape:  (173018, 11) 

Data shapes: (1557162, 115, 1) (1557162, 11) (173018, 115, 1) (173018, 11)
Evaluating for C:\Users\nalla\OneDrive\Provision_PT_838\dataset\Data...

Repeat:  1
Accuracy : 0.7705556647285253
F1 Score: 0.7366194988453987
Recall: 0.7705556647285253
Precision: 0.7532716839724021
Confusion Matrix: 
[[92027     0     0     0     0  6453     0     0     0     1    48]
 [    0     0     0     0     0     0     0  5643     0     0     0]
 [    0     0     0     0     0     2     0  2899     0     0     0]
 [    0     0     0     0     0     2     0  2817     0     0     0]
 [    0     0     0     0     0  8953     0     0     0     1     0]
 [    0     0     0     0     0 10408     0     0     0     0     0]
 [    0     0     0     0     0     0  2726     0     0  3402     1]
 [    0     0     0     0     0     0  

C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [3]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.utils import to_categorical
from numpy import mean, std, dstack
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score, precision_score

# load the dataset, returns train and test X and y elements
def load_dataset(path):
    # Load input data and output data
    print('\nLoading dataset...')
    files = os.listdir(path)
    dataX_files = [file for file in files if 'x_' in file]
    datay_files = [file for file in files if 'y_' in file]
    dataX_files.sort()
    datay_files.sort()

    dataX = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in dataX_files])
    
    datay = pd.DataFrame()  # Initialize datay as an empty DataFrame
    unique_label = 1
    for file in datay_files:
        datay_temp = pd.read_csv(path + '/' + file, delimiter=',')
        if (datay_temp.iloc[:, 0] == 1).any():  # Check if there are any 1 values in the first column
            datay_temp.loc[datay_temp.iloc[:, 0] == 1, datay_temp.columns[0]] = unique_label
            if datay.empty:  # If datay is empty, directly assign datay_temp
                datay = datay_temp
            else:
                datay = pd.concat([datay, datay_temp])  # Append only the modified datay_temp
            unique_label += 1

    # Split data into train and test sets
    trainX, testX = model_selection.train_test_split(dataX, test_size=0.10, random_state=42, shuffle=True)
    trainy, testy = model_selection.train_test_split(datay, test_size=0.10, random_state=42, shuffle=True)

    # Get 3D training data (assuming your data requires this step)
    listtrain = [trainX]
    listtest = [testX]
    trainX = dstack(listtrain)
    testX = dstack(listtest)
    print('\nAfter stacking...')
    print('trainX.shape: ', trainX.shape)
    print('testX.shape: ', testX.shape)

    # Calculate number of classes based on unique values in trainy
    num_classes = np.unique(trainy).shape[0]

    # Convert output data to categorical form
    trainy = to_categorical(trainy.values.ravel(), num_classes=num_classes)
    testy = to_categorical(testy.values.ravel(), num_classes=num_classes)
    print('\nAfter categorizing...')
    print('trainy.shape: ', trainy.shape)
    print('testy.shape: ', testy.shape, '\n')

    return trainX, trainy, testX, testy

from sklearn.metrics import f1_score, recall_score, confusion_matrix,precision_score
from sklearn.utils import class_weight
from sklearn.metrics import precision_score

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import AveragePooling1D

def evaluate_model(trainX, trainy, testX, testy):
    # Flatten the data for Random Forest
    trainX_flat = trainX.reshape(trainX.shape[0], -1)
    testX_flat = testX.reshape(testX.shape[0], -1)

    # Convert categorical labels back to 1-D for Random Forest
    trainy_flat = np.argmax(trainy, axis=1)
    testy_flat = np.argmax(testy, axis=1)

    # Define the model with depth 4
    model = RandomForestClassifier(n_estimators=100, max_depth=1)
    model.fit(trainX_flat, trainy_flat)

    # Predict the test set results
    y_pred = model.predict(testX_flat)

    # Calculate Accuracy, F1 score, Recall, and Precision
    accuracy = accuracy_score(testy_flat, y_pred)
    f1 = f1_score(testy_flat, y_pred, average='weighted')
    recall = recall_score(testy_flat, y_pred, average='weighted')
    precision = precision_score(testy_flat, y_pred, average='weighted')

    # Compute confusion matrix
    cm = confusion_matrix(testy_flat, y_pred)

    print(f'Accuracy : {accuracy}')
    print(f'F1 Score: {f1}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'Confusion Matrix: \n{cm}\n')       

    return accuracy, f1, recall, precision, cm
  
def run_experiment(repeats, datasetname):
    experiment_start_time = time.time()
    trainX, trainy, testX, testy = load_dataset(datasetname)
    print('Data shapes:', trainX.shape, trainy.shape, testX.shape, testy.shape)  # Debugging line
    print('Evaluating for ' + datasetname + '...\n')
    evaluation_start_time = time.time()
    scores = list()
    for r in range(repeats):
        print('Repeat: ', r + 1) 
        score = evaluate_model(trainX, trainy, testX, testy)
        print('Model output length:', len(score))  # Corrected line
        accuracy= score[0] * 100
        print('Score: %.2f' % accuracy, '\n')
        scores.append(score)
    print('Results for ' + datasetname + ':')
    
    print("Duration of loading data: %d seconds" % (evaluation_start_time - experiment_start_time))
    print("Duration of evaluating model: %d seconds" % (time.time() - evaluation_start_time))
    print("Duration of whole experiment: %d seconds" % (time.time() - experiment_start_time))

run_experiment(1, 'C:\\Users\\nalla\\OneDrive\\Provision_PT_838\\dataset\\Data')


Loading dataset...

After stacking...
trainX.shape:  (1557162, 115, 1)
testX.shape:  (173018, 115, 1)

After categorizing...
trainy.shape:  (1557162, 11)
testy.shape:  (173018, 11) 

Data shapes: (1557162, 115, 1) (1557162, 11) (173018, 115, 1) (173018, 11)
Evaluating for C:\Users\nalla\OneDrive\Provision_PT_838\dataset\Data...

Repeat:  1
Accuracy : 0.6189356020760846
F1 Score: 0.5089084770322791
Recall: 0.6189356020760846
Precision: 0.4351077233713771
Confusion Matrix: 
[[98529     0     0     0     0     0     0     0     0     0     0]
 [ 1158     0     0     0     0     0     0     0     0  4485     0]
 [   56     0     0     0     0     0     0     0     0  2845     0]
 [ 1769     0     0     0     0     0     0     0     0  1050     0]
 [ 8954     0     0     0     0     0     0     0     0     0     0]
 [10408     0     0     0     0     0     0     0     0     0     0]
 [ 2146     0     0     0     0     0     0     0     0  3983     0]
 [   66     0     0     0     0     0  

C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.utils import to_categorical
from numpy import mean, std, dstack
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score, precision_score

# load the dataset, returns train and test X and y elements
def load_dataset(path):
    # Load input data and output data
    print('\nLoading dataset...')
    files = os.listdir(path)
    dataX_files = [file for file in files if 'x_' in file]
    datay_files = [file for file in files if 'y_' in file]
    dataX_files.sort()
    datay_files.sort()

    dataX = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in dataX_files])
    
    datay = pd.DataFrame()  # Initialize datay as an empty DataFrame
    unique_label = 1
    for file in datay_files:
        datay_temp = pd.read_csv(path + '/' + file, delimiter=',')
        if (datay_temp.iloc[:, 0] == 1).any():  # Check if there are any 1 values in the first column
            datay_temp.loc[datay_temp.iloc[:, 0] == 1, datay_temp.columns[0]] = unique_label
            if datay.empty:  # If datay is empty, directly assign datay_temp
                datay = datay_temp
            else:
                datay = pd.concat([datay, datay_temp])  # Append only the modified datay_temp
            unique_label += 1

    # Split data into train and test sets
    trainX, testX = model_selection.train_test_split(dataX, test_size=0.10, random_state=42, shuffle=True)
    trainy, testy = model_selection.train_test_split(datay, test_size=0.10, random_state=42, shuffle=True)

    # Get 3D training data (assuming your data requires this step)
    listtrain = [trainX]
    listtest = [testX]
    trainX = dstack(listtrain)
    testX = dstack(listtest)
    print('\nAfter stacking...')
    print('trainX.shape: ', trainX.shape)
    print('testX.shape: ', testX.shape)

    # Calculate number of classes based on unique values in trainy
    num_classes = np.unique(trainy).shape[0]

    # Convert output data to categorical form
    trainy = to_categorical(trainy.values.ravel(), num_classes=num_classes)
    testy = to_categorical(testy.values.ravel(), num_classes=num_classes)
    print('\nAfter categorizing...')
    print('trainy.shape: ', trainy.shape)
    print('testy.shape: ', testy.shape, '\n')

    return trainX, trainy, testX, testy

from sklearn.metrics import f1_score, recall_score, confusion_matrix,precision_score
from sklearn.utils import class_weight
from sklearn.metrics import precision_score

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import AveragePooling1D

def evaluate_model(trainX, trainy, testX, testy):
    # Flatten the data for Random Forest
    trainX_flat = trainX.reshape(trainX.shape[0], -1)
    testX_flat = testX.reshape(testX.shape[0], -1)

    # Convert categorical labels back to 1-D for Random Forest
    trainy_flat = np.argmax(trainy, axis=1)
    testy_flat = np.argmax(testy, axis=1)

    # Define the model with depth 4
    model = RandomForestClassifier(n_estimators=100, max_depth=2)
    model.fit(trainX_flat, trainy_flat)

    # Predict the test set results
    y_pred = model.predict(testX_flat)

    # Calculate Accuracy, F1 score, Recall, and Precision
    accuracy = accuracy_score(testy_flat, y_pred)
    f1 = f1_score(testy_flat, y_pred, average='weighted')
    recall = recall_score(testy_flat, y_pred, average='weighted')
    precision = precision_score(testy_flat, y_pred, average='weighted')

    # Compute confusion matrix
    cm = confusion_matrix(testy_flat, y_pred)

    print(f'Accuracy : {accuracy}')
    print(f'F1 Score: {f1}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'Confusion Matrix: \n{cm}\n')       

    return accuracy, f1, recall, precision, cm
  
def run_experiment(repeats, datasetname):
    experiment_start_time = time.time()
    trainX, trainy, testX, testy = load_dataset(datasetname)
    print('Data shapes:', trainX.shape, trainy.shape, testX.shape, testy.shape)  # Debugging line
    print('Evaluating for ' + datasetname + '...\n')
    evaluation_start_time = time.time()
    scores = list()
    for r in range(repeats):
        print('Repeat: ', r + 1) 
        score = evaluate_model(trainX, trainy, testX, testy)
        print('Model output length:', len(score))  # Corrected line
        accuracy= score[0] * 100
        print('Score: %.2f' % accuracy, '\n')
        scores.append(score)
    print('Results for ' + datasetname + ':')
    
    print("Duration of loading data: %d seconds" % (evaluation_start_time - experiment_start_time))
    print("Duration of evaluating model: %d seconds" % (time.time() - evaluation_start_time))
    print("Duration of whole experiment: %d seconds" % (time.time() - experiment_start_time))

run_experiment(1, 'C:\\Users\\nalla\\OneDrive\\Provision_PT_838\\dataset\\Data')


Loading dataset...

After stacking...
trainX.shape:  (1557162, 115, 1)
testX.shape:  (173018, 115, 1)

After categorizing...
trainy.shape:  (1557162, 11)
testy.shape:  (173018, 11) 

Data shapes: (1557162, 115, 1) (1557162, 11) (173018, 115, 1) (173018, 11)
Evaluating for C:\Users\nalla\OneDrive\Provision_PT_838\dataset\Data...

Repeat:  1
Accuracy : 0.8195274480111896
F1 Score: 0.7579015889748837
Recall: 0.8195274480111896
Precision: 0.736694925783623
Confusion Matrix: 
[[98524     0     0     0     0     5     0     0     0     0     0]
 [   26  5545     0     0     0     0     0    72     0     0     0]
 [   25  2690     0     0     0     0     0   184     0     2     0]
 [ 1974     0     0     0     0     0     0   836     0     9     0]
 [   14     0     0     0     0  8940     0     0     0     0     0]
 [   11     0     0     0     0 10397     0     0     0     0     0]
 [    0     0     0     0     0     1     0    14     0  6114     0]
 [    0     0     0     0     0     0   

C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [1]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.utils import to_categorical
from numpy import mean, std, dstack
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score, precision_score

# load the dataset, returns train and test X and y elements
def load_dataset(path):
    # Load input data and output data
    print('\nLoading dataset...')
    files = os.listdir(path)
    dataX_files = [file for file in files if 'x_' in file]
    datay_files = [file for file in files if 'y_' in file]
    dataX_files.sort()
    datay_files.sort()

    dataX = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in dataX_files])
    
    datay = pd.DataFrame()  # Initialize datay as an empty DataFrame
    unique_label = 1
    for file in datay_files:
        datay_temp = pd.read_csv(path + '/' + file, delimiter=',')
        if (datay_temp.iloc[:, 0] == 1).any():  # Check if there are any 1 values in the first column
            datay_temp.loc[datay_temp.iloc[:, 0] == 1, datay_temp.columns[0]] = unique_label
            if datay.empty:  # If datay is empty, directly assign datay_temp
                datay = datay_temp
            else:
                datay = pd.concat([datay, datay_temp])  # Append only the modified datay_temp
            unique_label += 1

    # Split data into train and test sets
    trainX, testX = model_selection.train_test_split(dataX, test_size=0.10, random_state=42, shuffle=True)
    trainy, testy = model_selection.train_test_split(datay, test_size=0.10, random_state=42, shuffle=True)

    # Get 3D training data (assuming your data requires this step)
    listtrain = [trainX]
    listtest = [testX]
    trainX = dstack(listtrain)
    testX = dstack(listtest)
    print('\nAfter stacking...')
    print('trainX.shape: ', trainX.shape)
    print('testX.shape: ', testX.shape)

    # Calculate number of classes based on unique values in trainy
    num_classes = np.unique(trainy).shape[0]

    # Convert output data to categorical form
    trainy = to_categorical(trainy.values.ravel(), num_classes=num_classes)
    testy = to_categorical(testy.values.ravel(), num_classes=num_classes)
    print('\nAfter categorizing...')
    print('trainy.shape: ', trainy.shape)
    print('testy.shape: ', testy.shape, '\n')

    return trainX, trainy, testX, testy

from sklearn.metrics import f1_score, recall_score, confusion_matrix,precision_score
from sklearn.utils import class_weight
from sklearn.metrics import precision_score

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import AveragePooling1D

def evaluate_model(trainX, trainy, testX, testy):
    # Flatten the data for Random Forest
    trainX_flat = trainX.reshape(trainX.shape[0], -1)
    testX_flat = testX.reshape(testX.shape[0], -1)

    # Convert categorical labels back to 1-D for Random Forest
    trainy_flat = np.argmax(trainy, axis=1)
    testy_flat = np.argmax(testy, axis=1)

    # Define the model with depth 4
    model = RandomForestClassifier(n_estimators=100, max_depth=3)
    model.fit(trainX_flat, trainy_flat)

    # Predict the test set results
    y_pred = model.predict(testX_flat)

    # Calculate Accuracy, F1 score, Recall, and Precision
    accuracy = accuracy_score(testy_flat, y_pred)
    f1 = f1_score(testy_flat, y_pred, average='weighted')
    recall = recall_score(testy_flat, y_pred, average='weighted')
    precision = precision_score(testy_flat, y_pred, average='weighted')

    # Compute confusion matrix
    cm = confusion_matrix(testy_flat, y_pred)

    print(f'Accuracy : {accuracy}')
    print(f'F1 Score: {f1}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'Confusion Matrix: \n{cm}\n')       

    return accuracy, f1, recall, precision, cm
  
def run_experiment(repeats, datasetname):
    experiment_start_time = time.time()
    trainX, trainy, testX, testy = load_dataset(datasetname)
    print('Data shapes:', trainX.shape, trainy.shape, testX.shape, testy.shape)  # Debugging line
    print('Evaluating for ' + datasetname + '...\n')
    evaluation_start_time = time.time()
    scores = list()
    for r in range(repeats):
        print('Repeat: ', r + 1) 
        score = evaluate_model(trainX, trainy, testX, testy)
        print('Model output length:', len(score))  # Corrected line
        accuracy= score[0] * 100
        print('Score: %.2f' % accuracy, '\n')
        scores.append(score)
    print('Results for ' + datasetname + ':')
    
    print("Duration of loading data: %d seconds" % (evaluation_start_time - experiment_start_time))
    print("Duration of evaluating model: %d seconds" % (time.time() - evaluation_start_time))
    print("Duration of whole experiment: %d seconds" % (time.time() - experiment_start_time))

run_experiment(1, 'C:\\Users\\nalla\\OneDrive\\Provision_PT_838\\dataset\\Data')


Loading dataset...

After stacking...
trainX.shape:  (1557162, 115, 1)
testX.shape:  (173018, 115, 1)

After categorizing...
trainy.shape:  (1557162, 11)
testy.shape:  (173018, 11) 

Data shapes: (1557162, 115, 1) (1557162, 11) (173018, 115, 1) (173018, 11)
Evaluating for C:\Users\nalla\OneDrive\Provision_PT_838\dataset\Data...

Repeat:  1
Accuracy : 0.9607208498537725
F1 Score: 0.9518204415336554
Recall: 0.9607208498537725
Precision: 0.952289583201261
Confusion Matrix: 
[[98524     0     0     0     0     5     0     0     0     0     0]
 [    0  5629     0     0     0     0     0     0    14     0     0]
 [    2  2894     0     0     0     0     0     0     5     0     0]
 [  134     0     0  2679     0     0     0     0     6     0     0]
 [   14     0     0     0  8939     1     0     0     0     0     0]
 [   11     0     0     0     0 10397     0     0     0     0     0]
 [    0     0     0     0     1     0  3114   161     0  2514   339]
 [    0     0     0     0     0     0   

C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [2]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.utils import to_categorical
from numpy import mean, std, dstack
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score, precision_score

# load the dataset, returns train and test X and y elements
def load_dataset(path):
    # Load input data and output data
    print('\nLoading dataset...')
    files = os.listdir(path)
    dataX_files = [file for file in files if 'x_' in file]
    datay_files = [file for file in files if 'y_' in file]
    dataX_files.sort()
    datay_files.sort()

    dataX = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in dataX_files])
    
    datay = pd.DataFrame()  # Initialize datay as an empty DataFrame
    unique_label = 1
    for file in datay_files:
        datay_temp = pd.read_csv(path + '/' + file, delimiter=',')
        if (datay_temp.iloc[:, 0] == 1).any():  # Check if there are any 1 values in the first column
            datay_temp.loc[datay_temp.iloc[:, 0] == 1, datay_temp.columns[0]] = unique_label
            if datay.empty:  # If datay is empty, directly assign datay_temp
                datay = datay_temp
            else:
                datay = pd.concat([datay, datay_temp])  # Append only the modified datay_temp
            unique_label += 1

    # Split data into train and test sets
    trainX, testX = model_selection.train_test_split(dataX, test_size=0.10, random_state=42, shuffle=True)
    trainy, testy = model_selection.train_test_split(datay, test_size=0.10, random_state=42, shuffle=True)

    # Get 3D training data (assuming your data requires this step)
    listtrain = [trainX]
    listtest = [testX]
    trainX = dstack(listtrain)
    testX = dstack(listtest)
    print('\nAfter stacking...')
    print('trainX.shape: ', trainX.shape)
    print('testX.shape: ', testX.shape)

    # Calculate number of classes based on unique values in trainy
    num_classes = np.unique(trainy).shape[0]

    # Convert output data to categorical form
    trainy = to_categorical(trainy.values.ravel(), num_classes=num_classes)
    testy = to_categorical(testy.values.ravel(), num_classes=num_classes)
    print('\nAfter categorizing...')
    print('trainy.shape: ', trainy.shape)
    print('testy.shape: ', testy.shape, '\n')

    return trainX, trainy, testX, testy

from sklearn.metrics import f1_score, recall_score, confusion_matrix,precision_score
from sklearn.utils import class_weight
from sklearn.metrics import precision_score

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import AveragePooling1D

def evaluate_model(trainX, trainy, testX, testy):
    # Flatten the data for Random Forest
    trainX_flat = trainX.reshape(trainX.shape[0], -1)
    testX_flat = testX.reshape(testX.shape[0], -1)

    # Convert categorical labels back to 1-D for Random Forest
    trainy_flat = np.argmax(trainy, axis=1)
    testy_flat = np.argmax(testy, axis=1)

    # Define the model with depth 4
    model = RandomForestClassifier(n_estimators=100, max_depth=4)
    model.fit(trainX_flat, trainy_flat)

    # Predict the test set results
    y_pred = model.predict(testX_flat)

    # Calculate Accuracy, F1 score, Recall, and Precision
    accuracy = accuracy_score(testy_flat, y_pred)
    f1 = f1_score(testy_flat, y_pred, average='weighted')
    recall = recall_score(testy_flat, y_pred, average='weighted')
    precision = precision_score(testy_flat, y_pred, average='weighted')

    # Compute confusion matrix
    cm = confusion_matrix(testy_flat, y_pred)

    print(f'Accuracy : {accuracy}')
    print(f'F1 Score: {f1}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'Confusion Matrix: \n{cm}\n')       

    return accuracy, f1, recall, precision, cm
  
def run_experiment(repeats, datasetname):
    experiment_start_time = time.time()
    trainX, trainy, testX, testy = load_dataset(datasetname)
    print('Data shapes:', trainX.shape, trainy.shape, testX.shape, testy.shape)  # Debugging line
    print('Evaluating for ' + datasetname + '...\n')
    evaluation_start_time = time.time()
    scores = list()
    for r in range(repeats):
        print('Repeat: ', r + 1) 
        score = evaluate_model(trainX, trainy, testX, testy)
        print('Model output length:', len(score))  # Corrected line
        accuracy= score[0] * 100
        print('Score: %.2f' % accuracy, '\n')
        scores.append(score)
    print('Results for ' + datasetname + ':')
    
    print("Duration of loading data: %d seconds" % (evaluation_start_time - experiment_start_time))
    print("Duration of evaluating model: %d seconds" % (time.time() - evaluation_start_time))
    print("Duration of whole experiment: %d seconds" % (time.time() - experiment_start_time))

run_experiment(1, 'C:\\Users\\nalla\\OneDrive\\Provision_PT_838\\dataset\\Data')


Loading dataset...

After stacking...
trainX.shape:  (1557162, 115, 1)
testX.shape:  (173018, 115, 1)

After categorizing...
trainy.shape:  (1557162, 11)
testy.shape:  (173018, 11) 

Data shapes: (1557162, 115, 1) (1557162, 11) (173018, 115, 1) (173018, 11)
Evaluating for C:\Users\nalla\OneDrive\Provision_PT_838\dataset\Data...

Repeat:  1
Accuracy : 0.9834121305297715
F1 Score: 0.9772692298000288
Recall: 0.9834121305297715
Precision: 0.9888647288060544
Confusion Matrix: 
[[98528     0     0     0     0     1     0     0     0     0     0]
 [    0  5642     0     1     0     0     0     0     0     0     0]
 [    2  2827    72     0     0     0     0     0     0     0     0]
 [    2     0     0  2817     0     0     0     0     0     0     0]
 [    3     0     0     0  8950     1     0     0     0     0     0]
 [   10     0     0     0     0 10398     0     0     0     0     0]
 [    0     0     0     0     0     0  6107     0     0     3    19]
 [    0     0     0     0     0     0  

In [2]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.utils import to_categorical
from numpy import mean, std, dstack
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score, precision_score

# load the dataset, returns train and test X and y elements
def load_dataset(path):
    # Load input data and output data
    print('\nLoading dataset...')
    files = os.listdir(path)
    dataX_files = [file for file in files if 'x_' in file]
    datay_files = [file for file in files if 'y_' in file]
    dataX_files.sort()
    datay_files.sort()

    dataX = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in dataX_files])
    
    datay = pd.DataFrame()  # Initialize datay as an empty DataFrame
    unique_label = 1
    for file in datay_files:
        datay_temp = pd.read_csv(path + '/' + file, delimiter=',')
        if (datay_temp.iloc[:, 0] == 1).any():  # Check if there are any 1 values in the first column
            datay_temp.loc[datay_temp.iloc[:, 0] == 1, datay_temp.columns[0]] = unique_label
            if datay.empty:  # If datay is empty, directly assign datay_temp
                datay = datay_temp
            else:
                datay = pd.concat([datay, datay_temp])  # Append only the modified datay_temp
            unique_label += 1

    # Split data into train and test sets
    trainX, testX = model_selection.train_test_split(dataX, test_size=0.10, random_state=42, shuffle=True)
    trainy, testy = model_selection.train_test_split(datay, test_size=0.10, random_state=42, shuffle=True)

    # Get 3D training data (assuming your data requires this step)
    listtrain = [trainX]
    listtest = [testX]
    trainX = dstack(listtrain)
    testX = dstack(listtest)
    print('\nAfter stacking...')
    print('trainX.shape: ', trainX.shape)
    print('testX.shape: ', testX.shape)

    # Calculate number of classes based on unique values in trainy
    num_classes = np.unique(trainy).shape[0]

    # Convert output data to categorical form
    trainy = to_categorical(trainy.values.ravel(), num_classes=num_classes)
    testy = to_categorical(testy.values.ravel(), num_classes=num_classes)
    print('\nAfter categorizing...')
    print('trainy.shape: ', trainy.shape)
    print('testy.shape: ', testy.shape, '\n')

    return trainX, trainy, testX, testy

from sklearn.metrics import f1_score, recall_score, confusion_matrix,precision_score
from sklearn.utils import class_weight
from sklearn.metrics import precision_score

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import AveragePooling1D

def evaluate_model(trainX, trainy, testX, testy):
    # Flatten the data for Random Forest
    trainX_flat = trainX.reshape(trainX.shape[0], -1)
    testX_flat = testX.reshape(testX.shape[0], -1)

    # Convert categorical labels back to 1-D for Random Forest
    trainy_flat = np.argmax(trainy, axis=1)
    testy_flat = np.argmax(testy, axis=1)

    # Define the model with depth 5
    model = RandomForestClassifier(n_estimators=100, max_depth=5)
    model.fit(trainX_flat, trainy_flat)

    # Predict the test set results
    y_pred = model.predict(testX_flat)

    # Calculate Accuracy, F1 score, Recall, and Precision
    accuracy = accuracy_score(testy_flat, y_pred)
    f1 = f1_score(testy_flat, y_pred, average='weighted')
    recall = recall_score(testy_flat, y_pred, average='weighted')
    precision = precision_score(testy_flat, y_pred, average='weighted')

    # Compute confusion matrix
    cm = confusion_matrix(testy_flat, y_pred)

    print(f'Accuracy : {accuracy}')
    print(f'F1 Score: {f1}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'Confusion Matrix: \n{cm}\n')       

    return accuracy, f1, recall, precision, cm
  
def run_experiment(repeats, datasetname):
    experiment_start_time = time.time()
    trainX, trainy, testX, testy = load_dataset(datasetname)
    print('Data shapes:', trainX.shape, trainy.shape, testX.shape, testy.shape)  # Debugging line
    print('Evaluating for ' + datasetname + '...\n')
    evaluation_start_time = time.time()
    scores = list()
    for r in range(repeats):
        print('Repeat: ', r + 1) 
        score = evaluate_model(trainX, trainy, testX, testy)
        print('Model output length:', len(score))  # Corrected line
        accuracy= score[0] * 100
        print('Score: %.2f' % accuracy, '\n')
        scores.append(score)
    print('Results for ' + datasetname + ':')
    
    print("Duration of loading data: %d seconds" % (evaluation_start_time - experiment_start_time))
    print("Duration of evaluating model: %d seconds" % (time.time() - evaluation_start_time))
    print("Duration of whole experiment: %d seconds" % (time.time() - experiment_start_time))

run_experiment(1, 'C:\\Users\\nalla\\OneDrive\\Provision_PT_838\\dataset\\Data')


Loading dataset...

After stacking...
trainX.shape:  (1557162, 115, 1)
testX.shape:  (173018, 115, 1)

After categorizing...
trainy.shape:  (1557162, 11)
testy.shape:  (173018, 11) 

Data shapes: (1557162, 115, 1) (1557162, 11) (173018, 115, 1) (173018, 11)
Evaluating for C:\Users\nalla\OneDrive\Provision_PT_838\dataset\Data...

Repeat:  1
Accuracy : 0.9832618571478112
F1 Score: 0.9771168114332199
Recall: 0.9832618571478112
Precision: 0.9886957978297196
Confusion Matrix: 
[[98529     0     0     0     0     0     0     0     0     0     0]
 [    0  5634     0     9     0     0     0     0     0     0     0]
 [    2  2824    72     3     0     0     0     0     0     0     0]
 [    2     0     0  2817     0     0     0     0     0     0     0]
 [    3     0     0     0  8950     1     0     0     0     0     0]
 [   10     0     0     0     0 10398     0     0     0     0     0]
 [    0     0     0     0     0     0  6089     0     0     3    37]
 [    0     0     0     0     0     0  

In [ ]:
import matplotlib.pyplot as plt

# Data
x = [1, 2, 3, 4, 5]
y = [61.893, 81.952, 96.072, 98.341, 98.336]

# Create the plot
plt.plot(x, y, marker='o')

# Add title and labels
plt.title('Accuracy vs Max Depth')
plt.xlabel('Max Depth')
plt.ylabel('Accuracy')

# Show the plot
plt.show()


In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
import numpy as np
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.utils import to_categorical
from numpy import mean, std, dstack
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score, precision_score

# load the dataset, returns train and test X and y elements
def load_dataset(path):
    # Load input data and output data
    print('\nLoading dataset...')
    files = os.listdir(path)
    dataX_files = [file for file in files if 'x_' in file]
    datay_files = [file for file in files if 'y_' in file]
    dataX_files.sort()
    datay_files.sort()

    dataX = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in dataX_files])
    
    datay = pd.DataFrame()  # Initialize datay as an empty DataFrame
    unique_label = 1
    for file in datay_files:
        datay_temp = pd.read_csv(path + '/' + file, delimiter=',')
        if (datay_temp.iloc[:, 0] == 1).any():  # Check if there are any 1 values in the first column
            datay_temp.loc[datay_temp.iloc[:, 0] == 1, datay_temp.columns[0]] = unique_label
            if datay.empty:  # If datay is empty, directly assign datay_temp
                datay = datay_temp
            else:
                datay = pd.concat([datay, datay_temp])  # Append only the modified datay_temp
            unique_label += 1

    # Split data into train and test sets
    trainX, testX = model_selection.train_test_split(dataX, test_size=0.20, random_state=42, shuffle=True)
    trainy, testy = model_selection.train_test_split(datay, test_size=0.20, random_state=42, shuffle=True)

    # Get 3D training data (assuming your data requires this step)
    listtrain = [trainX]
    listtest = [testX]
    trainX = dstack(listtrain)
    testX = dstack(listtest)
    print('\nAfter stacking...')
    print('trainX.shape: ', trainX.shape)
    print('testX.shape: ', testX.shape)

    # Calculate number of classes based on unique values in trainy
    num_classes = np.unique(trainy).shape[0]

    # Convert output data to categorical form
    trainy = to_categorical(trainy.values.ravel(), num_classes=num_classes)
    testy = to_categorical(testy.values.ravel(), num_classes=num_classes)
    print('\nAfter categorizing...')
    print('trainy.shape: ', trainy.shape)
    print('testy.shape: ', testy.shape, '\n')

    return trainX, trainy, testX, testy

from sklearn.metrics import f1_score, recall_score, confusion_matrix,precision_score
from sklearn.utils import class_weight
from sklearn.metrics import precision_score

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import AveragePooling1D

def evaluate_model(trainX, trainy, testX, testy):
    # Flatten the data for Decision Tree
    trainX_flat = trainX.reshape(trainX.shape[0], -1)
    testX_flat = testX.reshape(testX.shape[0], -1)

    # Convert categorical labels back to 1-D for Decision Tree
    trainy_flat = np.argmax(trainy, axis=1)
    testy_flat = np.argmax(testy, axis=1)

    # Define the model
    model = DecisionTreeClassifier(max_depth=4)
    model.fit(trainX_flat, trainy_flat)

    # Predict the test set results
    y_pred = model.predict(testX_flat)

    # Calculate Accuracy, F1 score, Recall, and Precision
    accuracy = accuracy_score(testy_flat, y_pred)
    f1 = f1_score(testy_flat, y_pred, average='weighted')
    recall = recall_score(testy_flat, y_pred, average='weighted')
    precision = precision_score(testy_flat, y_pred, average='weighted')

    # Compute confusion matrix
    cm = confusion_matrix(testy_flat, y_pred)

    print(f'Accuracy : {accuracy}')
    print(f'F1 Score: {f1}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'Confusion Matrix: \n{cm}\n')       

    return accuracy, f1, recall, precision, cm
  
def run_experiment(repeats, datasetname):
    experiment_start_time = time.time()
    trainX, trainy, testX, testy = load_dataset(datasetname)
    print('Data shapes:', trainX.shape, trainy.shape, testX.shape, testy.shape)  # Debugging line
    print('Evaluating for ' + datasetname + '...\n')
    evaluation_start_time = time.time()
    scores = list()
    for r in range(repeats):
        print('Repeat: ', r + 1) 
        score = evaluate_model(trainX, trainy, testX, testy)
        print('Model output length:', len(score))  # Corrected line
        accuracy= score[0] * 100
        print('Score: %.2f' % accuracy, '\n')
        scores.append(score)
    print('Results for ' + datasetname + ':')
    
    print("Duration of loading data: %d seconds" % (evaluation_start_time - experiment_start_time))
    print("Duration of evaluating model: %d seconds" % (time.time() - evaluation_start_time))
    print("Duration of whole experiment: %d seconds" % (time.time() - experiment_start_time))

run_experiment(1, 'C:\\Users\\nalla\\OneDrive\\Provision_PT_838\\dataset\\Data')





Loading dataset...

After stacking...
trainX.shape:  (1384144, 115, 1)
testX.shape:  (346036, 115, 1)

After categorizing...
trainy.shape:  (1384144, 11)
testy.shape:  (346036, 11) 

Data shapes: (1384144, 115, 1) (1384144, 11) (346036, 115, 1) (346036, 11)
Evaluating for C:\Users\nalla\OneDrive\Provision_PT_838\dataset\Data...

Repeat:  1
Accuracy : 0.8801483082685039
F1 Score: 0.8321224962662769
Recall: 0.8801483082685039
Precision: 0.8017916817825582
Confusion Matrix: 
[[196835      0      0      0      0      0      0      0     53      0
       0]
 [ 11533      0      0      0      0      0      0      0      0      0
       0]
 [  5764      0      0      0      0      0      0      0      0      0
       0]
 [  5674      0      0      0      0      0      0      0      0      0
       0]
 [    23      0      0      0  17867      4      0      0      0      0
       0]
 [    18      0      0      0      0  20939      0      0      0      0
       0]
 [  7062      0      0      0 

C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
